# データサイエンス 100 本ノック（構造化データ加工編） - Python


## はじめに

- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas 等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません


In [1]:
# noqa
import os
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import math
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from imblearn.under_sampling import RandomUnderSampler
import polars as pl


dtype = {
    "customer_id": str,
    "gender_cd": str,
    "postal_cd": str,
    "application_store_cd": str,
    "status_cd": str,
    "category_major_cd": str,
    "category_medium_cd": str,
    "category_small_cd": str,
    "product_cd": str,
    "store_cd": str,
    "prefecture_cd": str,
    "tel_no": str,
    "street": str,
}


df_customer = pl.read_csv("../data/customer.csv")
df_category = pl.read_csv("../data/category.csv")
df_product = pl.read_csv("../data/product.csv")
df_receipt = pl.read_csv("../data/receipt.csv")
df_store = pl.read_csv("../data/store.csv")
df_geocode = pl.read_csv("../data/geocode.csv")


# cast column types
for df in [df_customer, df_category, df_product, df_receipt, df_store, df_geocode]:
    for col, col_type in dtype.items():
        if col in df.columns:
            df = df.select(pl.col(col).cast(pl.String).alias(col))

# 演習問題


---

> P-001: レシート明細データ（df_receipt）から全項目の先頭 10 件を表示し、どのようなデータを保有しているか目視で確認せよ。


In [2]:
df_receipt.head(10)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001""","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097""","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014""","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000""","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050""","""P060102007""",1,90
20190605,1559692800,"""S13003""",1112,1,"""CS003515000195""","""P050102002""",1,138
20181205,1543968000,"""S14024""",1102,2,"""CS024514000042""","""P080101005""",1,30
20190922,1569110400,"""S14040""",1102,1,"""CS040415000178""","""P070501004""",1,128
20170504,1493856000,"""S13020""",1112,2,"""ZZ000000000000""","""P071302010""",1,770


---

> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客 ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10 件表示せよ。


In [3]:
df_receipt.select(
    pl.col("sales_ymd"), pl.col("customer_id"), pl.col("product_cd"), pl.col("amount")
).head(10)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20181103,"""CS006214000001""","""P070305012""",158
20181118,"""CS008415000097""","""P070701017""",81
20170712,"""CS028414000014""","""P060101005""",170
20190205,"""ZZ000000000000""","""P050301001""",25
20180821,"""CS025415000050""","""P060102007""",90
20190605,"""CS003515000195""","""P050102002""",138
20181205,"""CS024514000042""","""P080101005""",30
20190922,"""CS040415000178""","""P070501004""",128
20170504,"""ZZ000000000000""","""P071302010""",770


---

> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客 ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10 件表示せよ。ただし、sales_ymd を sales_date に項目名を変更して抽出すること。


In [4]:
df_receipt.select(
    pl.col("sales_ymd").alias("sales_date"),
    pl.col("customer_id"),
    pl.col("product_cd"),
    pl.col("amount"),
).head(10)

sales_date,customer_id,product_cd,amount
i64,str,str,i64
20181103,"""CS006214000001""","""P070305012""",158
20181118,"""CS008415000097""","""P070701017""",81
20170712,"""CS028414000014""","""P060101005""",170
20190205,"""ZZ000000000000""","""P050301001""",25
20180821,"""CS025415000050""","""P060102007""",90
20190605,"""CS003515000195""","""P050102002""",138
20181205,"""CS024514000042""","""P080101005""",30
20190922,"""CS040415000178""","""P070501004""",128
20170504,"""ZZ000000000000""","""P071302010""",770


---

> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客 ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
>
> - 顧客 ID（customer_id）が"CS018205000001"


In [5]:
df_receipt.select(
    pl.col("sales_ymd"), pl.col("customer_id"), pl.col("product_cd"), pl.col("amount")
).filter(pl.col("customer_id") == "CS018205000001")

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20180414,"""CS018205000001""","""P060104007""",600
20170614,"""CS018205000001""","""P050206001""",990
20170614,"""CS018205000001""","""P060702015""",108
20190216,"""CS018205000001""","""P071005024""",102
…,…,…,…
20190924,"""CS018205000001""","""P060805001""",495
20190226,"""CS018205000001""","""P071401020""",2200
20180911,"""CS018205000001""","""P071401005""",1100


---

> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客 ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
>
> - 顧客 ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が 1,000 以上


In [ ]:
df_receipt.select(
    pl.col("sales_ymd"), pl.col("customer_id"), pl.col("product_cd"), pl.col("amount")
).filter((pl.col("customer_id") == "CS018205000001") & (pl.col("amount") >= 1000))

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20190226,"""CS018205000001""","""P071401020""",2200
20180911,"""CS018205000001""","""P071401005""",1100


---

> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客 ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
>
> - 顧客 ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が 1,000 以上または売上数量（quantity）が 5 以上


In [ ]:
df_receipt.select(
    pl.col("sales_ymd"),
    pl.col("customer_id"),
    pl.col("product_cd"),
    pl.col("amount"),
    pl.col("quantity"),
).filter(
    (pl.col("customer_id") == "CS018205000001")
    & ((pl.col("amount") >= 1000) | (pl.col("quantity") >= 5))
)

sales_ymd,customer_id,product_cd,amount,quantity
i64,str,str,i64,i64
20180911,"""CS018205000001""","""P071401012""",2200,1
20180414,"""CS018205000001""","""P060104007""",600,6
20170614,"""CS018205000001""","""P050206001""",990,5
20190226,"""CS018205000001""","""P071401020""",2200,1
20180911,"""CS018205000001""","""P071401005""",1100,1


---

> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客 ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
>
> - 顧客 ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が 1,000 以上 2,000 以下


In [ ]:
df_receipt.select(
    pl.col("sales_ymd"), pl.col("customer_id"), pl.col("product_cd"), pl.col("amount")
).filter(
    (pl.col("customer_id") == "CS018205000001")
    & (pl.col("amount") >= 1000)
    & (pl.col("amount") <= 2000)
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401005""",1100


---

> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客 ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
>
> - 顧客 ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外


In [ ]:
df_receipt.select(
    pl.col("sales_ymd"), pl.col("customer_id"), pl.col("product_cd"), pl.col("amount")
).filter(
    (pl.col("customer_id") == "CS018205000001") & (pl.col("product_cd") != "P071401019")
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20180414,"""CS018205000001""","""P060104007""",600
20170614,"""CS018205000001""","""P050206001""",990
20170614,"""CS018205000001""","""P060702015""",108
20190216,"""CS018205000001""","""P071005024""",102
…,…,…,…
20190924,"""CS018205000001""","""P060805001""",495
20190226,"""CS018205000001""","""P071401020""",2200
20180911,"""CS018205000001""","""P071401005""",1100


---

> P-009: 以下の処理において、出力結果を変えずに OR を AND に書き換えよ。
>
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`


In [ ]:
df_store.filter((pl.col("prefecture_cd") != 13) & (pl.col("floor_area") <= 900))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,i64,str,str,str,str,f64,f64,f64
"""S14046""","""北山田店""",14,"""神奈川県""","""神奈川県横浜市都筑区北山田一丁目""","""カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ""","""045-123-4049""",139.5916,35.56189,831.0
"""S14011""","""日吉本町店""",14,"""神奈川県""","""神奈川県横浜市港北区日吉本町四丁目""","""カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ""","""045-123-4033""",139.6316,35.54655,890.0
"""S12013""","""習志野店""",12,"""千葉県""","""千葉県習志野市芝園一丁目""","""チバケンナラシノシシバゾノイッチョウメ""","""047-123-4002""",140.022,35.66122,808.0


---

> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10 件表示せよ。


In [18]:
df_store.filter(pl.col("store_cd").str.starts_with("S14")).head(10)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,i64,str,str,str,str,f64,f64,f64
"""S14010""","""菊名店""",14,"""神奈川県""","""神奈川県横浜市港北区菊名一丁目""","""カナガワケンヨコハマシコウホククキクナイッチョウメ""","""045-123-4032""",139.6326,35.50049,1732.0
"""S14033""","""阿久和店""",14,"""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4043""",139.4961,35.45918,1495.0
"""S14036""","""相模原中央店""",14,"""神奈川県""","""神奈川県相模原市中央二丁目""","""カナガワケンサガミハラシチュウオウニチョウメ""","""042-123-4045""",139.3716,35.57327,1679.0
"""S14040""","""長津田店""",14,"""神奈川県""","""神奈川県横浜市緑区長津田みなみ台五丁目""","""カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ""","""045-123-4046""",139.4994,35.52398,1548.0
"""S14050""","""阿久和西店""",14,"""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4053""",139.4961,35.45918,1830.0
"""S14028""","""二ツ橋店""",14,"""神奈川県""","""神奈川県横浜市瀬谷区二ツ橋町""","""カナガワケンヨコハマシセヤクフタツバシチョウ""","""045-123-4042""",139.4963,35.46304,1574.0
"""S14012""","""本牧和田店""",14,"""神奈川県""","""神奈川県横浜市中区本牧和田""","""カナガワケンヨコハマシナカクホンモクワダ""","""045-123-4034""",139.6582,35.42156,1341.0
"""S14046""","""北山田店""",14,"""神奈川県""","""神奈川県横浜市都筑区北山田一丁目""","""カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ""","""045-123-4049""",139.5916,35.56189,831.0
"""S14022""","""逗子店""",14,"""神奈川県""","""神奈川県逗子市逗子一丁目""","""カナガワケンズシシズシイッチョウメ""","""046-123-4036""",139.5789,35.29642,1838.0


---

> P-011: 顧客データ（df_customer）から顧客 ID（customer_id）の末尾が 1 のものだけ全項目抽出し、10 件表示せよ。


In [19]:
df_customer.filter(pl.col("customer_id").str.ends_with("1")).head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,i64,str,str,i64,str,str,str,i64,str
"""CS037613000071""","""六角 雅彦""",9,"""不明""","""1952-04-01""",66,"""136-0076""","""東京都江東区南砂**********""","""S13037""",20150414,"""0-00000000-0"""
"""CS028811000001""","""堀井 かおり""",1,"""女性""","""1933-03-27""",86,"""245-0016""","""神奈川県横浜市泉区和泉町**********""","""S14028""",20160115,"""0-00000000-0"""
"""CS040412000191""","""川井 郁恵""",1,"""女性""","""1977-01-05""",42,"""226-0021""","""神奈川県横浜市緑区北八朔町**********""","""S14040""",20151101,"""1-20091025-4"""
"""CS028314000011""","""小菅 あおい""",1,"""女性""","""1983-11-26""",35,"""246-0038""","""神奈川県横浜市瀬谷区宮沢**********""","""S14028""",20151123,"""1-20080426-5"""
"""CS039212000051""","""藤島 恵梨香""",1,"""女性""","""1997-02-03""",22,"""166-0001""","""東京都杉並区阿佐谷北**********""","""S13039""",20171121,"""1-20100215-4"""
"""CS015412000111""","""松居 奈月""",1,"""女性""","""1972-10-04""",46,"""136-0071""","""東京都江東区亀戸**********""","""S13015""",20150629,"""0-00000000-0"""
"""CS004702000041""","""野島 洋""",0,"""男性""","""1943-08-24""",75,"""176-0022""","""東京都練馬区向山**********""","""S13004""",20170218,"""0-00000000-0"""
"""CS041515000001""","""栗田 千夏""",1,"""女性""","""1967-01-02""",52,"""206-0001""","""東京都多摩市和田**********""","""S13041""",20160422,"""E-20100803-F"""
"""CS029313000221""","""北条 ひかり""",1,"""女性""","""1987-06-19""",31,"""279-0011""","""千葉県浦安市美浜**********""","""S12029""",20180810,"""0-00000000-0"""


---

> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。


In [ ]:
df_store.filter(pl.col("address").str.contains("横浜市"))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,i64,str,str,str,str,f64,f64,f64
"""S14010""","""菊名店""",14,"""神奈川県""","""神奈川県横浜市港北区菊名一丁目""","""カナガワケンヨコハマシコウホククキクナイッチョウメ""","""045-123-4032""",139.6326,35.50049,1732.0
"""S14033""","""阿久和店""",14,"""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4043""",139.4961,35.45918,1495.0
"""S14040""","""長津田店""",14,"""神奈川県""","""神奈川県横浜市緑区長津田みなみ台五丁目""","""カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ""","""045-123-4046""",139.4994,35.52398,1548.0
"""S14050""","""阿久和西店""",14,"""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4053""",139.4961,35.45918,1830.0
"""S14028""","""二ツ橋店""",14,"""神奈川県""","""神奈川県横浜市瀬谷区二ツ橋町""","""カナガワケンヨコハマシセヤクフタツバシチョウ""","""045-123-4042""",139.4963,35.46304,1574.0
…,…,…,…,…,…,…,…,…,…
"""S14046""","""北山田店""",14,"""神奈川県""","""神奈川県横浜市都筑区北山田一丁目""","""カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ""","""045-123-4049""",139.5916,35.56189,831.0
"""S14011""","""日吉本町店""",14,"""神奈川県""","""神奈川県横浜市港北区日吉本町四丁目""","""カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ""","""045-123-4033""",139.6316,35.54655,890.0
"""S14048""","""中川中央店""",14,"""神奈川県""","""神奈川県横浜市都筑区中川中央二丁目""","""カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ""","""045-123-4051""",139.5758,35.54912,1657.0


---

> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットの A〜F で始まるデータを全項目抽出し、10 件表示せよ。


In [22]:
# > P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットの A〜F で始まるデータを全項目抽出し、10 件表示せよ。
df_customer.filter(pl.col("status_cd").str.contains("^[A-F]")).head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,i64,str,str,i64,str,str,str,i64,str
"""CS031415000172""","""宇多田 貴美子""",1,"""女性""","""1976-10-04""",42,"""151-0053""","""東京都渋谷区代々木**********""","""S13031""",20150529,"""D-20100325-C"""
"""CS015414000103""","""奥野 陽子""",1,"""女性""","""1977-08-09""",41,"""136-0073""","""東京都江東区北砂**********""","""S13015""",20150722,"""B-20100609-B"""
"""CS011215000048""","""芦田 沙耶""",1,"""女性""","""1992-02-01""",27,"""223-0062""","""神奈川県横浜市港北区日吉本町**********""","""S14011""",20150228,"""C-20100421-9"""
"""CS029415000023""","""梅田 里穂""",1,"""女性""","""1976-01-17""",43,"""279-0043""","""千葉県浦安市富士見**********""","""S12029""",20150610,"""D-20100918-E"""
"""CS035415000029""","""寺沢 真希""",9,"""不明""","""1977-09-27""",41,"""158-0096""","""東京都世田谷区玉川台**********""","""S13035""",20141220,"""F-20101029-F"""
"""CS031415000106""","""宇野 由美子""",1,"""女性""","""1970-02-26""",49,"""151-0053""","""東京都渋谷区代々木**********""","""S13031""",20150201,"""F-20100511-E"""
"""CS029215000025""","""石倉 美帆""",1,"""女性""","""1993-09-28""",25,"""279-0022""","""千葉県浦安市今川**********""","""S12029""",20150708,"""B-20100820-C"""
"""CS033605000005""","""猪股 雄太""",0,"""男性""","""1955-12-05""",63,"""246-0031""","""神奈川県横浜市瀬谷区瀬谷**********""","""S14033""",20150425,"""F-20100917-E"""
"""CS033415000229""","""板垣 菜々美""",1,"""女性""","""1977-11-07""",41,"""246-0021""","""神奈川県横浜市瀬谷区二ツ橋町**********""","""S14033""",20150712,"""F-20100326-E"""


---

> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の 1〜9 で終わるデータを全項目抽出し、10 件表示せよ。


In [23]:
df_customer.filter(pl.col("status_cd").str.contains("[1-9]$")).head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,i64,str,str,i64,str,str,str,i64,str
"""CS001215000145""","""田崎 美紀""",1,"""女性""","""1995-03-29""",24,"""144-0055""","""東京都大田区仲六郷**********""","""S13001""",20170605,"""6-20090929-2"""
"""CS033513000180""","""安斎 遥""",1,"""女性""","""1962-07-11""",56,"""241-0823""","""神奈川県横浜市旭区善部町**********""","""S14033""",20150728,"""6-20080506-5"""
"""CS011215000048""","""芦田 沙耶""",1,"""女性""","""1992-02-01""",27,"""223-0062""","""神奈川県横浜市港北区日吉本町**********""","""S14011""",20150228,"""C-20100421-9"""
"""CS040412000191""","""川井 郁恵""",1,"""女性""","""1977-01-05""",42,"""226-0021""","""神奈川県横浜市緑区北八朔町**********""","""S14040""",20151101,"""1-20091025-4"""
"""CS009315000023""","""皆川 文世""",1,"""女性""","""1980-04-15""",38,"""154-0012""","""東京都世田谷区駒沢**********""","""S13009""",20150319,"""5-20080322-1"""
"""CS015315000033""","""福士 璃奈子""",1,"""女性""","""1983-03-17""",36,"""135-0043""","""東京都江東区塩浜**********""","""S13015""",20141024,"""4-20080219-3"""
"""CS023513000066""","""神戸 そら""",1,"""女性""","""1961-12-17""",57,"""210-0005""","""神奈川県川崎市川崎区東田町**********""","""S14023""",20150915,"""5-20100524-9"""
"""CS035513000134""","""市川 美帆""",1,"""女性""","""1960-03-27""",59,"""156-0053""","""東京都世田谷区桜**********""","""S13035""",20150227,"""8-20100711-9"""
"""CS001515000263""","""高松 夏空""",1,"""女性""","""1962-11-09""",56,"""144-0051""","""東京都大田区西蒲田**********""","""S13001""",20160812,"""1-20100804-1"""


---

> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットの A〜F で始まり、末尾が数字の 1〜9 で終わるデータを全項目抽出し、10 件表示せよ。


In [25]:
df_customer.filter(pl.col("status_cd").str.contains("^[A-F].*[1-9]$")).head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,i64,str,str,i64,str,str,str,i64,str
"""CS011215000048""","""芦田 沙耶""",1,"""女性""","""1992-02-01""",27,"""223-0062""","""神奈川県横浜市港北区日吉本町**********""","""S14011""",20150228,"""C-20100421-9"""
"""CS022513000105""","""島村 貴美子""",1,"""女性""","""1962-03-12""",57,"""249-0002""","""神奈川県逗子市山の根**********""","""S14022""",20150320,"""A-20091115-7"""
"""CS001515000096""","""水野 陽子""",9,"""不明""","""1960-11-29""",58,"""144-0053""","""東京都大田区蒲田本町**********""","""S13001""",20150614,"""A-20100724-7"""
"""CS013615000053""","""西脇 季衣""",1,"""女性""","""1953-10-18""",65,"""261-0026""","""千葉県千葉市美浜区幕張西**********""","""S12013""",20150128,"""B-20100329-6"""
"""CS020412000161""","""小宮 薫""",1,"""女性""","""1974-05-21""",44,"""174-0042""","""東京都板橋区東坂下**********""","""S13020""",20150822,"""B-20081021-3"""
"""CS001215000097""","""竹中 あさみ""",1,"""女性""","""1990-07-25""",28,"""146-0095""","""東京都大田区多摩川**********""","""S13001""",20170315,"""A-20100211-2"""
"""CS035212000007""","""内村 恵梨香""",1,"""女性""","""1990-12-04""",28,"""152-0023""","""東京都目黒区八雲**********""","""S13035""",20151013,"""B-20101018-6"""
"""CS002515000386""","""野田 コウ""",1,"""女性""","""1963-05-30""",55,"""185-0013""","""東京都国分寺市西恋ケ窪**********""","""S13002""",20160410,"""C-20100127-8"""
"""CS001615000372""","""稲垣 寿々花""",1,"""女性""","""1956-10-29""",62,"""144-0035""","""東京都大田区南蒲田**********""","""S13001""",20170403,"""A-20100104-1"""


---

> P-016: 店舗データ（df_store）から、電話番号（tel_no）が 3 桁-3 桁-4 桁のデータを全項目表示せよ。


In [26]:
df_store.filter(pl.col("tel_no").str.contains(r"^\d{3}-\d{3}-\d{4}$"))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,i64,str,str,str,str,f64,f64,f64
"""S12014""","""千草台店""",12,"""千葉県""","""千葉県千葉市稲毛区千草台一丁目""","""チバケンチバシイナゲクチグサダイイッチョウメ""","""043-123-4003""",140.118,35.63559,1698.0
"""S13002""","""国分寺店""",13,"""東京都""","""東京都国分寺市本多二丁目""","""トウキョウトコクブンジシホンダニチョウメ""","""042-123-4008""",139.4802,35.70566,1735.0
"""S14010""","""菊名店""",14,"""神奈川県""","""神奈川県横浜市港北区菊名一丁目""","""カナガワケンヨコハマシコウホククキクナイッチョウメ""","""045-123-4032""",139.6326,35.50049,1732.0
"""S14033""","""阿久和店""",14,"""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4043""",139.4961,35.45918,1495.0
"""S14036""","""相模原中央店""",14,"""神奈川県""","""神奈川県相模原市中央二丁目""","""カナガワケンサガミハラシチュウオウニチョウメ""","""042-123-4045""",139.3716,35.57327,1679.0
…,…,…,…,…,…,…,…,…,…
"""S14045""","""厚木店""",14,"""神奈川県""","""神奈川県厚木市中町二丁目""","""カナガワケンアツギシナカチョウニチョウメ""","""046-123-4048""",139.3651,35.44182,980.0
"""S12029""","""東野店""",12,"""千葉県""","""千葉県浦安市東野一丁目""","""チバケンウラヤスシヒガシノイッチョウメ""","""047-123-4004""",139.8968,35.65086,1101.0
"""S12053""","""高洲店""",12,"""千葉県""","""千葉県浦安市高洲五丁目""","""チバケンウラヤスシタカスゴチョウメ""","""047-123-4006""",139.9176,35.63755,1555.0


---

> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を 10 件表示せよ。


In [29]:
df_customer.sort("birth_day").head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,i64,str,str,i64,str,str,str,i64,str
"""CS003813000014""","""村山 菜々美""",1,"""女性""","""1928-11-26""",90,"""182-0007""","""東京都調布市菊野台**********""","""S13003""",20160214,"""0-00000000-0"""
"""CS026813000004""","""吉村 朝陽""",1,"""女性""","""1928-12-14""",90,"""251-0043""","""神奈川県藤沢市辻堂元町**********""","""S14026""",20150723,"""0-00000000-0"""
"""CS018811000003""","""熊沢 美里""",1,"""女性""","""1929-01-07""",90,"""204-0004""","""東京都清瀬市野塩**********""","""S13018""",20150403,"""0-00000000-0"""
"""CS027803000004""","""内村 拓郎""",0,"""男性""","""1929-01-12""",90,"""251-0031""","""神奈川県藤沢市鵠沼藤が谷**********""","""S14027""",20151227,"""0-00000000-0"""
"""CS013801000003""","""天野 拓郎""",0,"""男性""","""1929-01-15""",90,"""274-0824""","""千葉県船橋市前原東**********""","""S12013""",20160120,"""0-00000000-0"""
"""CS001814000022""","""鶴田 里穂""",1,"""女性""","""1929-01-28""",90,"""144-0045""","""東京都大田区南六郷**********""","""S13001""",20161012,"""A-20090415-7"""
"""CS016815000002""","""山元 美紀""",1,"""女性""","""1929-02-22""",90,"""184-0005""","""東京都小金井市桜町**********""","""S13016""",20150629,"""C-20090923-C"""
"""CS009815000003""","""中田 里穂""",1,"""女性""","""1929-04-08""",89,"""154-0014""","""東京都世田谷区新町**********""","""S13009""",20150421,"""D-20091021-E"""
"""CS012813000013""","""宇野 南朋""",1,"""女性""","""1929-04-09""",89,"""231-0806""","""神奈川県横浜市中区本牧町**********""","""S14012""",20150712,"""0-00000000-0"""


---

> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を 10 件表示せよ。


In [31]:
df_customer.sort("birth_day", descending=True).head(10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,i64,str,str,i64,str,str,str,i64,str
"""CS035114000004""","""大村 美里""",1,"""女性""","""2007-11-25""",11,"""156-0053""","""東京都世田谷区桜**********""","""S13035""",20150619,"""6-20091205-6"""
"""CS022103000002""","""福山 はじめ""",9,"""不明""","""2007-10-02""",11,"""249-0006""","""神奈川県逗子市逗子**********""","""S14022""",20160909,"""0-00000000-0"""
"""CS002113000009""","""柴田 真悠子""",1,"""女性""","""2007-09-17""",11,"""184-0014""","""東京都小金井市貫井南町**********""","""S13002""",20160304,"""0-00000000-0"""
"""CS004115000014""","""松井 京子""",1,"""女性""","""2007-08-09""",11,"""165-0031""","""東京都中野区上鷺宮**********""","""S13004""",20161120,"""1-20081231-1"""
"""CS002114000010""","""山内 遥""",1,"""女性""","""2007-06-03""",11,"""184-0015""","""東京都小金井市貫井北町**********""","""S13002""",20160920,"""6-20100510-1"""
"""CS025115000002""","""小柳 夏希""",1,"""女性""","""2007-04-18""",11,"""245-0018""","""神奈川県横浜市泉区上飯田町**********""","""S14025""",20160116,"""D-20100913-D"""
"""CS002113000025""","""広末 まなみ""",1,"""女性""","""2007-03-30""",12,"""184-0015""","""東京都小金井市貫井北町**********""","""S13002""",20171030,"""0-00000000-0"""
"""CS033112000003""","""長野 美紀""",1,"""女性""","""2007-03-22""",12,"""245-0051""","""神奈川県横浜市戸塚区名瀬町**********""","""S14033""",20150606,"""0-00000000-0"""
"""CS007115000006""","""福岡 瞬""",1,"""女性""","""2007-03-10""",12,"""285-0845""","""千葉県佐倉市西志津**********""","""S12007""",20151118,"""F-20101016-F"""


---

> P-019: レシート明細データ（df_receipt）に対し、1 件あたりの売上金額（amount）が高い順にランクを付与し、先頭から 10 件表示せよ。項目は顧客 ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。


In [40]:
df_receipt.select(
    pl.col("customer_id"),
    pl.col("amount"),
    pl.col("amount").rank(method="min", descending=True).alias("amount_rank"),
).sort(pl.col("amount_rank")).head(10)

customer_id,amount,amount_rank
str,i64,u32
"""CS011415000006""",10925,1
"""ZZ000000000000""",6800,2
"""CS028605000002""",5780,3
"""CS015515000034""",5480,4
"""ZZ000000000000""",5480,4
"""ZZ000000000000""",5480,4
"""ZZ000000000000""",5440,7
"""CS021515000089""",5440,7
"""CS015515000083""",5280,9


---

> P-020: レシート明細データ（df_receipt）に対し、1 件あたりの売上金額（amount）が高い順にランクを付与し、先頭から 10 件表示せよ。項目は顧客 ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。


In [4]:
df_receipt.select(
    pl.col("customer_id"),
    pl.col("amount"),
    pl.col("amount").rank(method="ordinal", descending=True).alias("amount_rank"),
).sort(pl.col("amount_rank")).head(10)

customer_id,amount,amount_rank
str,i64,u32
"""CS011415000006""",10925,1
"""ZZ000000000000""",6800,2
"""CS028605000002""",5780,3
"""CS015515000034""",5480,4
"""ZZ000000000000""",5480,5
"""ZZ000000000000""",5480,6
"""ZZ000000000000""",5440,7
"""CS021515000089""",5440,8
"""CS015515000083""",5280,9


---

> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。


In [6]:
df_receipt.shape[0]

104681

---

> P-022: レシート明細データ（df_receipt）の顧客 ID（customer_id）に対し、ユニーク件数をカウントせよ。


In [10]:
df_receipt.select(pl.col("customer_id")).n_unique()

8307

---

> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。


In [21]:
df_receipt.select(
    pl.col("store_cd"),
    pl.col("amount"),
    pl.col("quantity"),
).group_by(pl.col("store_cd")).sum()

store_cd,amount,quantity
str,i64,i64
"""S14023""",727630,2258
"""S13051""",107452,354
"""S13001""",811936,2347
"""S14049""",230808,788
"""S13038""",708884,2337
…,…,…
"""S13043""",587895,1881
"""S14034""",653681,2024
"""S13005""",629876,2004


---

> P-024: レシート明細データ（df_receipt）に対し、顧客 ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10 件表示せよ。


In [ ]:
df_receipt.select(
    pl.col("customer_id"),
    pl.col("sales_ymd"),
).group_by(pl.col("customer_id")).max()

customer_id,sales_ymd
str,i64
"""CS004612000146""",20171230
"""CS001114000005""",20190731
"""CS004314000027""",20190420
"""CS019314000045""",20180515
"""CS002415000510""",20190622
…,…
"""CS015515000165""",20190804
"""CS003512000501""",20181227
"""CS033304000004""",20191019


---

> P-025: レシート明細データ（df_receipt）に対し、顧客 ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10 件表示せよ。


In [27]:
df_receipt.select(
    pl.col("customer_id"),
    pl.col("sales_ymd"),
).group_by(pl.col("customer_id")).min()

customer_id,sales_ymd
str,i64
"""CS018604000001""",20170510
"""CS005412000036""",20171021
"""CS011515000234""",20170102
"""CS034513000122""",20170722
"""CS023615000057""",20171223
…,…
"""CS033512000155""",20171208
"""CS012415000178""",20170313
"""CS032413000073""",20170317


---

> P-026: レシート明細データ（df_receipt）に対し、顧客 ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを 10 件表示せよ。


In [33]:
df_receipt.select(
    pl.col("customer_id"),
    pl.col("sales_ymd"),
).group_by(pl.col("customer_id")).agg(
    pl.col("sales_ymd").max().alias("max_sales_ymd"),
    pl.col("sales_ymd").min().alias("min_sales_ymd"),
).filter(pl.col("max_sales_ymd") != pl.col("min_sales_ymd")).head(10)

customer_id,max_sales_ymd,min_sales_ymd
str,i64,i64
"""CS034513000239""",20190906,20180608
"""CS026605000002""",20190917,20170212
"""CS040411000020""",20190113,20170808
"""CS029215000031""",20190511,20170319
"""CS024515000069""",20190729,20171205
"""CS016214000008""",20190612,20170108
"""CS015315000140""",20180218,20171018
"""CS004415000700""",20190317,20181002
"""CS035513000089""",20191003,20170429


---

> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順で TOP5 を表示せよ。


In [39]:
df_receipt.select(
    pl.col("store_cd"),
    pl.col("amount"),
).group_by(pl.col("store_cd")).mean().sort("amount", descending=True)

store_cd,amount
str,f64
"""S13052""",402.86747
"""S13015""",351.11196
"""S13003""",350.915519
"""S14010""",348.791262
"""S13001""",348.470386
…,…
"""S13037""",299.907832
"""S14049""",297.816774
"""S14050""",290.591304


---

> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順で TOP5 を表示せよ。


In [43]:
df_receipt.select(
    pl.col("store_cd"),
    pl.col("amount"),
).group_by(pl.col("store_cd")).median().sort("amount", descending=True).head(5)

store_cd,amount
str,f64
"""S13052""",190.0
"""S14010""",188.0
"""S14050""",185.0
"""S13003""",180.0
"""S14040""",180.0


---

> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10 件表示させよ。


In [53]:
df_receipt.select(
    pl.col("store_cd"),
    pl.col("product_cd"),
).group_by(pl.col("store_cd")).agg(
    pl.col("product_cd").mode().first().alias("mode_product_cd")
).head(10)

store_cd,mode_product_cd
str,str
"""S13002""","""P060303001"""
"""S13004""","""P060303001"""
"""S14045""","""P060303001"""
"""S14022""","""P060303001"""
"""S14046""","""P060303001"""
"""S13020""","""P071401001"""
"""S13031""","""P060303001"""
"""S14028""","""P060303001"""
"""S13003""","""P071401001"""


---

> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で 5 件表示せよ。


In [ ]:
df_receipt.select(
    pl.col("store_cd"),
    pl.col("amount"),
).group_by(pl.col("store_cd")).agg(
    pl.col("amount").var(ddof=0).alias("amount_var")
).sort("amount_var", descending=True).head(5)

store_cd,amount_var
str,f64
"""S13052""",440088.701311
"""S14011""",306314.558164
"""S14034""",296920.081011
"""S13001""",295431.993329
"""S13015""",295294.361116


---

> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で 5 件表示せよ。


TIPS:

Pandas と Numpy で ddof のデフォルト値が異なることに注意しましょう

```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```


In [64]:
df_receipt.select(
    pl.col("store_cd"),
    pl.col("amount"),
).group_by(pl.col("store_cd")).agg(
    pl.col("amount").std(ddof=0).alias("amount_std")
).sort("amount_std", descending=True).head(5)

store_cd,amount_std
str,f64
"""S13052""",663.391816
"""S14011""",553.456916
"""S14034""",544.903736
"""S13001""",543.536561
"""S13015""",543.409938


---

> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。


In [79]:
df_receipt.select(
    pl.col("amount").quantile(p).alias(f"q{p}") for p in [0.25, 0.5, 0.75, 1]
).head(1)

q0.25,q0.5,q0.75,q1
f64,f64,f64,f64
102.0,170.0,288.0,10925.0


---

> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330 以上のものを抽出せよ。


In [83]:
df_receipt.select(
    pl.col("store_cd"),
    pl.col("amount"),
).group_by(pl.col("store_cd")).agg(
    pl.col("amount").mean().alias("amount_mean"),
).filter(pl.col("amount_mean") >= 330)

store_cd,amount_mean
str,f64
"""S14047""",330.077073
"""S13001""",348.470386
"""S14026""",332.340588
"""S13019""",330.208616
"""S14045""",330.082073
…,…
"""S14011""",335.718333
"""S13020""",337.879932
"""S13015""",351.11196


---

> P-034: レシート明細データ（df_receipt）に対し、顧客 ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客 ID が"Z"から始まるものは非会員を表すため、除外して計算すること。


In [10]:
df_receipt.select(
    pl.col("customer_id"),
    pl.col("amount"),
).filter(pl.col("customer_id").str.starts_with("Z").not_()).group_by(
    pl.col("customer_id")
).agg(
    pl.col("amount").sum().alias("amount_mean"),
).mean()

customer_id,amount_mean
str,f64
null,2547.742235


---

> P-035: レシート明細データ（df_receipt）に対し、顧客 ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10 件表示せよ。ただし、顧客 ID が"Z"から始まるものは非会員を表すため、除外して計算すること。


In [ ]:
df_receipt.select(
    pl.col("customer_id"),
    pl.col("amount"),
).filter(pl.col("customer_id").str.starts_with("Z").not_()).group_by(
    pl.col("customer_id")
).agg(
    pl.col("amount").sum().alias("amount_sum"),
).filter(pl.col("amount_sum") >= pl.col("amount_sum").mean()).head(10)

customer_id,amount_sum
str,i64
"""CS029414000112""",2741
"""CS019415000084""",3881
"""CS019414000072""",2724
"""CS016513000031""",4024
"""CS014215000052""",2919
"""CS008515000153""",3175
"""CS040415000035""",13119
"""CS029515000188""",7123
"""CS031515000036""",5383


---

> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を 10 件表示せよ。


In [2]:
df_receipt.join(
    df_store.select(pl.col("store_cd"), pl.col("store_name")),
    on="store_cd",
    how="inner",
).head(10)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
i64,i64,str,i64,i64,str,str,i64,i64,str
20181103,1541203200,"""S14006""",112,1,"""CS006214000001""","""P070305012""",1,158,"""葛が谷店"""
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097""","""P070701017""",1,81,"""成城店"""
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014""","""P060101005""",1,170,"""二ツ橋店"""
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000""","""P050301001""",1,25,"""新山下店"""
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050""","""P060102007""",1,90,"""大和店"""
20190605,1559692800,"""S13003""",1112,1,"""CS003515000195""","""P050102002""",1,138,"""狛江店"""
20181205,1543968000,"""S14024""",1102,2,"""CS024514000042""","""P080101005""",1,30,"""三田店"""
20190922,1569110400,"""S14040""",1102,1,"""CS040415000178""","""P070501004""",1,128,"""長津田店"""
20170504,1493856000,"""S13020""",1112,2,"""ZZ000000000000""","""P071302010""",1,770,"""十条仲原店"""


---

> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を 10 件表示せよ。


In [8]:
df_product.join(
    df_category.select(pl.col("category_small_cd"), pl.col("category_major_name")),
    on="category_small_cd",
    how="inner",
).head(10)

﻿product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name
str,i64,i64,i64,i64,i64,str
"""P040101001""",4,401,40101,198,149,"""惣菜"""
"""P040101002""",4,401,40101,218,164,"""惣菜"""
"""P040101003""",4,401,40101,230,173,"""惣菜"""
"""P040101004""",4,401,40101,248,186,"""惣菜"""
"""P040101005""",4,401,40101,268,201,"""惣菜"""
"""P040101006""",4,401,40101,298,224,"""惣菜"""
"""P040101007""",4,401,40101,338,254,"""惣菜"""
"""P040101008""",4,401,40101,420,315,"""惣菜"""
"""P040101009""",4,401,40101,498,374,"""惣菜"""


---

> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10 件表示せよ。ただし、売上実績がない顧客については売上金額を 0 として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客 ID が"Z"から始まるもの）は除外すること。


---

> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位 20 件を抽出したデータと、売上金額合計の多い顧客の上位 20 件を抽出したデータをそれぞれ作成し、さらにその 2 つを完全外部結合せよ。ただし、非会員（顧客 ID が"Z"から始まるもの）は除外すること。


---

> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。


In [35]:
df_store.select("store_cd").with_columns(pl.lit(1).alias("one")).join(
    df_product.select("category_major_cd").with_columns(pl.lit(1).alias("one")),
    on="one",
    how="full",
).shape[0]

531590

---

> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を 10 件表示せよ。


In [46]:
df_receipt.select(pl.col("sales_ymd"), pl.col("amount")).group_by(
    pl.col("sales_ymd")
).sum().sort("sales_ymd").with_columns(
    pl.col("sales_ymd").shift(1).alias("prev_sales_ymd"),
    pl.col("amount").shift(1).alias("prev_amount"),
).with_columns((pl.col("amount") - pl.col("prev_amount")).alias("diff_amount")).head(10)

sales_ymd,amount,prev_sales_ymd,prev_amount,diff_amount
i64,i64,i64,i64,i64
20170101,33723,null,null,null
20170102,24165,20170101,33723,-9558
20170103,27503,20170102,24165,3338
20170104,36165,20170103,27503,8662
20170105,37830,20170104,36165,1665
20170106,32387,20170105,37830,-5443
20170107,23415,20170106,32387,-8972
20170108,24737,20170107,23415,1322
20170109,26718,20170108,24737,1981


---

> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3 回前に売上があった日のデータを結合せよ。そして結果を 10 件表示せよ。


In [21]:
df_receipt.select(pl.col("sales_ymd"), pl.col("amount")).group_by(
    pl.col("sales_ymd")
).sum().sort("sales_ymd").with_columns(
    pl.col("sales_ymd").shift(1).alias("prev_sales_ymd"),
    pl.col("amount").shift(1).alias("prev_amount"),
    pl.col("sales_ymd").shift(2).alias("prev_sales_ymd2"),
    pl.col("amount").shift(2).alias("prev_amount2"),
    pl.col("sales_ymd").shift(3).alias("prev_sales_ymd3"),
    pl.col("amount").shift(3).alias("prev_amount3"),
)

sales_ymd,amount,prev_sales_ymd,prev_amount,prev_sales_ymd2,prev_amount2,prev_sales_ymd3,prev_amount3
i64,i64,i64,i64,i64,i64,i64,i64
20170101,33723,null,null,null,null,null,null
20170102,24165,20170101,33723,null,null,null,null
20170103,27503,20170102,24165,20170101,33723,null,null
20170104,36165,20170103,27503,20170102,24165,20170101,33723
20170105,37830,20170104,36165,20170103,27503,20170102,24165
…,…,…,…,…,…,…,…
20191027,37484,20191026,51771,20191025,28833,20191024,31868
20191028,40161,20191027,37484,20191026,51771,20191025,28833
20191029,36091,20191028,40161,20191027,37484,20191026,51771


---

> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（age から計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは 0 が男性、1 が女性、9 が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の 4 項目とすること（縦に年代、横に性別のクロス集計）。また、年代は 10 歳ごとの階級とすること。


In [ ]:
df_sales_summary = (
    df_receipt.join(df_customer, on="customer_id", how="inner")
    .select(
        pl.col("gender_cd"),
        (pl.col("age") // 10 * 10).alias("age"),
        pl.col("amount"),
    )
    .group_by(pl.col("gender_cd"), pl.col("age"))
    .agg(
        pl.col("amount").sum().alias("amount_mean"),
    )
    .sort(by="age")
    .pivot(on="gender_cd", index="age", values="amount_mean")
    .rename({"0": "Male", "1": "Female", "9": "Unknown"})
)
df_sales_summary

age,Male,Unknown,Female
i64,i64,i64,i64
10,1591,4317,149836
20,72940,44328,1363724
30,177322,50441,693047
40,19355,483512,9320791
50,54320,342923,6685192
60,272469,71418,987741
70,13435,2427,29764
80,46360,5111,262923
90,null,null,6260


---

> P-044： 043 で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の 3 項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。


In [ ]:
df_sales_summary.rename({"Male": "00", "Female": "01", "Unknown": "99"}).unpivot(
    index=["age"], value_name="amount"
).sort(by="age")

age,variable,amount
i64,str,i64
10,"""00""",1591
10,"""99""",4317
10,"""01""",149836
20,"""00""",72940
20,"""99""",44328
…,…,…
80,"""99""",5111
80,"""01""",262923
90,"""00""",null


---

> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これを YYYYMMDD 形式の文字列に変換し、顧客 ID（customer_id）とともに 10 件表示せよ。


In [ ]:
df_customer = df_customer.with_columns(
    pl.col("birth_day").cast(pl.Date).alias("birth_day")
)
df_customer.select(
    pl.col("customer_id"),
    pl.col("birth_day").dt.strftime("%Y%m%d").alias("birth_day"),
).head(10)

birth_day,customer_id
str,str
"""19810429""","""CS021313000114"""
"""19520401""","""CS037613000071"""
"""19761004""","""CS031415000172"""
"""19330327""","""CS028811000001"""
"""19950329""","""CS001215000145"""
"""19740915""","""CS020401000016"""
"""19770809""","""CS015414000103"""
"""19730817""","""CS029403000008"""
"""19310502""","""CS015804000004"""


---

> P-046: 顧客データ（df_customer）の申し込み日（application_date）は YYYYMMDD 形式の文字列型でデータを保有している。これを日付型に変換し、顧客 ID（customer_id）とともに 10 件表示せよ。


In [85]:
df_customer.select(
    pl.col("customer_id"),
    pl.col("birth_day").cast(pl.Date).alias("birth_day"),
)

customer_id,birth_day
str,date
"""CS021313000114""",1981-04-29
"""CS037613000071""",1952-04-01
"""CS031415000172""",1976-10-04
"""CS028811000001""",1933-03-27
"""CS001215000145""",1995-03-29
…,…
"""CS002512000474""",1959-10-12
"""CS029414000065""",1970-10-19
"""CS012403000043""",1972-12-16


---

> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）は YYYYMMDD 形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに 10 件表示せよ。


In [90]:
df_receipt.select(
    pl.col("sales_ymd").cast(pl.String).str.to_date("%Y%m%d").alias("sales_date"),
    pl.col("receipt_no"),
    pl.col("receipt_sub_no"),
)

sales_date,receipt_no,receipt_sub_no
date,i64,i64
2018-11-03,112,1
2018-11-18,1132,2
2017-07-12,1102,1
2019-02-05,1132,1
2018-08-21,1102,2
…,…,…
2018-02-21,1132,2
2019-09-11,1132,2
2017-03-11,1122,1


---

> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型の UNIX 秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに 10 件表示せよ。


In [101]:
df_receipt.select(
    pl.from_epoch(pl.col("sales_epoch")).cast(pl.Date).alias("sales_date"),
    pl.col("receipt_no"),
    pl.col("receipt_sub_no"),
).head(10)

sales_date,receipt_no,receipt_sub_no
date,i64,i64
2018-11-03,112,1
2018-11-18,1132,2
2017-07-12,1102,1
2019-02-05,1132,1
2018-08-21,1102,2
2019-06-05,1112,1
2018-12-05,1102,2
2019-09-22,1102,1
2017-05-04,1112,2


---

> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに 10 件表示せよ。


In [102]:
df_receipt.select(
    pl.from_epoch(pl.col("sales_epoch")).cast(pl.Date).alias("sales_date").dt.year(),
    pl.col("receipt_no"),
    pl.col("receipt_sub_no"),
).head(10)

sales_date,receipt_no,receipt_sub_no
i32,i64,i64
2018,112,1
2018,1132,2
2017,1102,1
2019,1132,1
2018,1102,2
2019,1112,1
2018,1102,2
2019,1102,1
2017,1112,2


---

> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに 10 件表示せよ。なお、「月」は 0 埋め 2 桁で取り出すこと。


In [ ]:
df_receipt.select(
    pl.from_epoch(pl.col("sales_epoch"))
    .cast(pl.Date)
    .alias("sales_date")
    .dt.month()
    .cast(pl.String)
    .str.zfill(2),
    pl.col("receipt_no"),
    pl.col("receipt_sub_no"),
).head(10)

sales_date,receipt_no,receipt_sub_no
str,i64,i64
"""11""",112,1
"""11""",1132,2
"""07""",1102,1
"""02""",1132,1
"""08""",1102,2
"""06""",1112,1
"""12""",1102,2
"""09""",1102,1
"""05""",1112,2


---

> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに 10 件表示せよ。なお、「日」は 0 埋め 2 桁で取り出すこと。


In [115]:
df_receipt.select(
    pl.from_epoch(pl.col("sales_epoch"))
    .cast(pl.Date)
    .alias("sales_date")
    .dt.day()
    .cast(pl.String)
    .str.zfill(2),
    pl.col("receipt_no"),
    pl.col("receipt_sub_no"),
).head(10)

sales_date,receipt_no,receipt_sub_no
str,i64,i64
"""03""",112,1
"""18""",1132,2
"""12""",1102,1
"""05""",1132,1
"""21""",1102,2
"""05""",1112,1
"""05""",1102,2
"""22""",1102,1
"""04""",1112,2


---

> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客 ID（customer_id）ごとに合計の上、売上金額合計に対して 2,000 円以下を 0、2,000 円より大きい金額を 1 に二値化し、顧客 ID、売上金額合計とともに 10 件表示せよ。ただし、顧客 ID が"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [124]:
df_receipt.group_by(pl.col("customer_id")).sum().select(
    pl.col("customer_id"),
    pl.col("amount").alias("sum_amount"),
    pl.when(pl.col("amount") >= 2000).then(1).otherwise(0).alias("is_high_amount"),
).head(10)

customer_id,sum_amount,is_high_amount
str,i64,i32
"""CS018414000039""",5955,1
"""CS024514000005""",979,0
"""CS031515000204""",3128,1
"""CS011115000005""",585,0
"""CS033512000066""",446,0
"""CS019513000064""",2365,1
"""CS019215000046""",8140,1
"""CS005415000086""",3859,1
"""CS024415000186""",5352,1


---

> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭 3 桁が 100〜209 のもの）を 1、それ以外のものを 0 に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。


In [ ]:
df_customer.select(
    pl.col("customer_id"),
    pl.col("postal_cd").str.slice(0, 3),
    pl.when(
        (pl.col("postal_cd").str.slice(0, 3) >= "100")
        & (pl.col("postal_cd").str.slice(0, 3) <= "209")
    )
    .then(1)
    .otherwise(0)
    .alias("is_tokyo"),
).join(
    df_receipt.select(pl.col("customer_id")),
    on="customer_id",
    how="inner",
).group_by(
    pl.col("is_tokyo"),
).agg(
    pl.col("customer_id").n_unique().alias("n_customers"),
).sort(by="is_tokyo")

is_tokyo,n_customers
i32,u32
0,3906
1,4400


---

> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客 ID、住所とともに 10 件表示せよ。値は埼玉県を 11、千葉県を 12、東京都を 13、神奈川県を 14 とすること。


In [ ]:
df_customer.select(
    pl.col("customer_id"),
    pl.col("address"),
    pl.col("address")
    .str.slice(0, 3)
    .replace(
        {
            "埼玉県": 11,
            "千葉県": 12,
            "東京都": 13,
            "神奈川": 14,
        }
    )
    .alias("pref_cd"),
)

customer_id,address,pref_cd
str,str,str
"""CS021313000114""","""神奈川県伊勢原市粟窪**********""","""14"""
"""CS037613000071""","""東京都江東区南砂**********""","""13"""
"""CS031415000172""","""東京都渋谷区代々木**********""","""13"""
"""CS028811000001""","""神奈川県横浜市泉区和泉町**********""","""14"""
"""CS001215000145""","""東京都大田区仲六郷**********""","""13"""
…,…,…
"""CS002512000474""","""東京都国分寺市光町**********""","""13"""
"""CS029414000065""","""千葉県浦安市富士見**********""","""12"""
"""CS012403000043""","""神奈川県横浜市中区本牧間門**********""","""14"""


---

> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客 ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客 ID、売上金額合計とともに 10 件表示せよ。カテゴリ値は順に 1〜4 とする。
>
> - 最小値以上第 1 四分位未満 ・・・ 1 を付与
> - 第 1 四分位以上第 2 四分位未満 ・・・ 2 を付与
> - 第 2 四分位以上第 3 四分位未満 ・・・ 3 を付与
> - 第 3 四分位以上 ・・・ 4 を付与


In [94]:
df_receipt.select(
    pl.col("customer_id"),
    pl.col("amount"),
).group_by(pl.col("customer_id")).agg(
    pl.col("amount").sum().alias("sum_amount"),
).with_columns(
    pl.when(pl.col("sum_amount") < pl.col("sum_amount").quantile(0.25))
    .then(1)
    .when(pl.col("sum_amount") < pl.col("sum_amount").quantile(0.5))
    .then(2)
    .when(pl.col("sum_amount") < pl.col("sum_amount").quantile(0.75))
    .then(3)
    .otherwise(4)
    .alias("amount_quantile")
)

customer_id,sum_amount,amount_quantile
str,i64,i32
"""CS004515000002""",258,1
"""CS039214000007""",6132,4
"""CS019305000009""",969,2
"""CS007512000113""",956,2
"""CS041414000001""",8007,4
…,…,…
"""CS018615000088""",2365,3
"""CS013615000030""",573,2
"""CS037614000008""",2317,3


---

> P-056: 顧客データ（df_customer）の年齢（age）をもとに 10 歳刻みで年代を算出し、顧客 ID（customer_id）、生年月日（birth_day）とともに 10 件表示せよ。ただし、60 歳以上は全て 60 歳代とすること。年代を表すカテゴリ名は任意とする。


In [102]:
df_customer.select(
    pl.col("customer_id"),
    pl.col("birth_day"),
    pl.col("age"),
).with_columns(
    pl.when(pl.col("age") >= 60)
    .then(60)
    .otherwise(pl.col("age") // 10 * 10)
    .alias("age")
).head(10)

customer_id,birth_day,age
str,str,i64
"""CS021313000114""","""1981-04-29""",30
"""CS037613000071""","""1952-04-01""",60
"""CS031415000172""","""1976-10-04""",40
"""CS028811000001""","""1933-03-27""",60
"""CS001215000145""","""1995-03-29""",20
"""CS020401000016""","""1974-09-15""",40
"""CS015414000103""","""1977-08-09""",40
"""CS029403000008""","""1973-08-17""",40
"""CS015804000004""","""1931-05-02""",60


---

> P-057: 056 の抽出結果と性別コード（gender_cd）により、新たに性別 × 年代の組み合わせを表すカテゴリデータを作成し、10 件表示せよ。組み合わせを表すカテゴリの値は任意とする。


In [ ]:
df_customer.select(
    pl.col("customer_id"),
    pl.col("birth_day"),
    pl.col("gender"),
    pl.col("age"),
).with_columns(
    pl.when(pl.col("age") >= 60)
    .then(60)
    .otherwise(pl.col("age") // 10 * 10)
    .alias("age_group")
).with_columns(
    pl.concat_str(
        pl.col("gender"), pl.col("age_group").cast(pl.String).str.zfill(2)
    ).alias("gender_age_group")
)

customer_id,birth_day,gender,age,age_group,gender_age_group
str,str,str,i64,i64,str
"""CS021313000114""","""1981-04-29""","""女性""",37,30,"""女性30"""
"""CS037613000071""","""1952-04-01""","""不明""",66,60,"""不明60"""
"""CS031415000172""","""1976-10-04""","""女性""",42,40,"""女性40"""
"""CS028811000001""","""1933-03-27""","""女性""",86,60,"""女性60"""
"""CS001215000145""","""1995-03-29""","""女性""",24,20,"""女性20"""
…,…,…,…,…,…
"""CS002512000474""","""1959-10-12""","""女性""",59,50,"""女性50"""
"""CS029414000065""","""1970-10-19""","""女性""",48,40,"""女性40"""
"""CS012403000043""","""1972-12-16""","""男性""",46,40,"""男性40"""


---

> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客 ID（customer_id）とともに 10 件表示せよ。


In [117]:
df_customer.select(
    pl.col("customer_id"),
    pl.col("gender_cd"),
).to_dummies(["gender_cd"]).head(10)

customer_id,gender_cd_0,gender_cd_1,gender_cd_9
str,u8,u8,u8
"""CS021313000114""",0,1,0
"""CS037613000071""",0,0,1
"""CS031415000172""",0,1,0
"""CS028811000001""",0,1,0
"""CS001215000145""",0,1,0
"""CS020401000016""",1,0,0
"""CS015414000103""",0,1,0
"""CS029403000008""",1,0,0
"""CS015804000004""",1,0,0


---

> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客 ID（customer_id）ごとに合計し、売上金額合計を平均 0、標準偏差 1 に標準化して顧客 ID、売上金額合計とともに 10 件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客 ID が"Z"から始まるのものは非会員を表すため、除外して計算すること。


TIPS:

- query()の引数 engine で'python'か'numexpr'かを選択でき、デフォルトはインストールされていれば numexpr が、無ければ python が使われます。さらに、文字列メソッドは engine='python'でないと query()内で使えません。


In [2]:
df_receipt.filter(pl.col("customer_id").str.starts_with("Z").not_()).group_by(
    pl.col("customer_id")
).agg(
    pl.col("amount").sum().alias("sum_amount"),
).with_columns(
    (
        (pl.col("sum_amount") - pl.col("sum_amount").mean())
        / pl.col("sum_amount").std(ddof=0)
    ).alias("z_score")
).sort(pl.col("customer_id"))

customer_id,sum_amount,z_score
str,i64,f64
"""CS001113000004""",1298,-0.459378
"""CS001114000005""",626,-0.70639
"""CS001115000010""",3044,0.182413
"""CS001205000004""",1988,-0.205749
"""CS001205000006""",3337,0.290114
…,…,…
"""CS051212000001""",336,-0.812988
"""CS051513000004""",551,-0.733959
"""CS051515000002""",265,-0.839086


---

> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客 ID（customer_id）ごとに合計し、売上金額合計を最小値 0、最大値 1 に正規化して顧客 ID、売上金額合計とともに 10 件表示せよ。ただし、顧客 ID が"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [ ]:
non_z_customer = pl.col("customer_id").str.starts_with("Z").not_()


df_receipt.filter(non_z_customer).group_by(pl.col("customer_id")).agg(
    pl.col("amount").sum().alias("sum_amount"),
).with_columns(
    (
        (pl.col("sum_amount") - pl.col("sum_amount").min())
        / (pl.col("sum_amount").max() - pl.col("sum_amount").min())
    ).alias("z_score")
).sort(pl.col("customer_id")).head(10)

customer_id,sum_amount,z_score
str,i64,f64
"""CS001113000004""",1298,0.053354
"""CS001114000005""",626,0.024157
"""CS001115000010""",3044,0.129214
"""CS001205000004""",1988,0.083333
"""CS001205000006""",3337,0.141945
"""CS001211000025""",456,0.016771
"""CS001212000027""",448,0.016423
"""CS001212000031""",296,0.009819
"""CS001212000046""",228,0.006865


---

> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客 ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底 10）して顧客 ID、売上金額合計とともに 10 件表示せよ。ただし、顧客 ID が"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [ ]:
df_receipt.filter(non_z_customer).group_by(pl.col("customer_id")).agg(
    pl.col("amount").sum().alias("sum_amount"),
).with_columns((pl.col("sum_amount").log10()).alias("z_score")).sort(
    pl.col("customer_id")
).head(10)

customer_id,sum_amount,z_score
str,i64,f64
"""CS001113000004""",1298,3.113275
"""CS001114000005""",626,2.796574
"""CS001115000010""",3044,3.483445
"""CS001205000004""",1988,3.298416
"""CS001205000006""",3337,3.523356
"""CS001211000025""",456,2.658965
"""CS001212000027""",448,2.651278
"""CS001212000031""",296,2.471292
"""CS001212000046""",228,2.357935


---

> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客 ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底 e）して顧客 ID、売上金額合計とともに 10 件表示せよ。ただし、顧客 ID が"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [23]:
df_receipt.filter(non_z_customer).group_by(pl.col("customer_id")).agg(
    pl.col("amount").sum().alias("sum_amount"),
).with_columns((pl.col("sum_amount").log()).alias("z_score")).sort(
    pl.col("customer_id")
).head(10)

customer_id,sum_amount,z_score
str,i64,f64
"""CS001113000004""",1298,7.16858
"""CS001114000005""",626,6.43935
"""CS001115000010""",3044,8.020928
"""CS001205000004""",1988,7.594884
"""CS001205000006""",3337,8.112827
"""CS001211000025""",456,6.122493
"""CS001212000027""",448,6.104793
"""CS001212000031""",296,5.690359
"""CS001212000046""",228,5.429346


---

> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を 10 件表示せよ。


In [27]:
df_product.with_columns(
    (pl.col("unit_price") - pl.col("unit_cost")).alias("unit_profit")
).head(10)

﻿product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_profit
str,i64,i64,i64,i64,i64,i64
"""P040101001""",4,401,40101,198,149,49
"""P040101002""",4,401,40101,218,164,54
"""P040101003""",4,401,40101,230,173,57
"""P040101004""",4,401,40101,248,186,62
"""P040101005""",4,401,40101,268,201,67
"""P040101006""",4,401,40101,298,224,74
"""P040101007""",4,401,40101,338,254,84
"""P040101008""",4,401,40101,420,315,105
"""P040101009""",4,401,40101,498,374,124


---

> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。


In [5]:
df_product.select(
    pl.col("unit_price"),
    pl.col("unit_cost"),
).with_columns(
    ((pl.col("unit_price") - pl.col("unit_cost")) / pl.col("unit_price")).alias(
        "unit_profit"
    )
).mean().item(0, -1)

0.24911389885177004

---

> P-065: 商品データ（df_product）の各商品について、利益率が 30%となる新たな単価を求めよ。ただし、1 円未満は切り捨てること。そして結果を 10 件表示させ、利益率がおよそ 30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。


In [19]:
formula = (pl.col("unit_cost") / 0.7).alias("new_price").floor()

df_product.select(pl.col("unit_cost")).with_columns(
    formula.cast(pl.UInt32),
).with_columns(
    ((pl.col("new_price") - pl.col("unit_cost")) / pl.col("new_price")).alias(
        "new_profit"
    ),
).head()

unit_cost,new_price,new_profit
i64,u32,f64
149,212,0.29717
164,234,0.299145
173,247,0.299595
186,265,0.298113
201,287,0.299652


---

> P-066: 商品データ（df_product）の各商品について、利益率が 30%となる新たな単価を求めよ。今回は、1 円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を 10 件表示させ、利益率がおよそ 30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。


In [20]:
formula = (pl.col("unit_cost") / 0.7).alias("new_price").round()

df_product.select(pl.col("unit_cost")).with_columns(
    formula.cast(pl.UInt32),
).with_columns(
    ((pl.col("new_price") - pl.col("unit_cost")) / pl.col("new_price")).alias(
        "new_profit"
    ),
).head()

unit_cost,new_price,new_profit
i64,u32,f64
149,213,0.300469
164,234,0.299145
173,247,0.299595
186,266,0.300752
201,287,0.299652


---

> P-067: 商品データ（df_product）の各商品について、利益率が 30%となる新たな単価を求めよ。今回は、1 円未満を切り上げること。そして結果を 10 件表示させ、利益率がおよそ 30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。


In [21]:
formula = (pl.col("unit_cost") / 0.7).alias("new_price").ceil()

df_product.select(pl.col("unit_cost")).with_columns(
    formula.cast(pl.UInt32),
).with_columns(
    ((pl.col("new_price") - pl.col("unit_cost")) / pl.col("new_price")).alias(
        "new_profit"
    ),
).head()

unit_cost,new_price,new_profit
i64,u32,f64
149,213,0.300469
164,235,0.302128
173,248,0.302419
186,266,0.300752
201,288,0.302083


---

> P-068: 商品データ（df_product）の各商品について、消費税率 10％の税込み金額を求めよ。1 円未満の端数は切り捨てとし、結果を 10 件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。


In [23]:
df_product.select(pl.col("unit_price")).with_columns(
    pl.col("unit_price").mul(1.1).floor().alias("with_tax")
)

unit_price,with_tax
i64,f64
198,217.0
218,239.0
230,253.0
248,272.0
268,294.0
…,…
280,308.0
680,748.0
1080,1188.0


---

> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を 10 件表示せよ。


In [57]:
if "\ufeffproduct_cd" in df_product.columns:  # BOM
    df_product = df_product.rename({"\ufeffproduct_cd": "product_cd"})

df_receipt.join(df_product, on="product_cd", how="inner").group_by(
    pl.col("customer_id")
).agg(
    pl.col("amount").sum().alias("sum_amount"),
    pl.when(pl.col("category_major_cd") == 7)
    .then(pl.col("amount"))
    .otherwise(0)
    .sum()
    .alias("sum_amount_cat7"),
).head()

customer_id,sum_amount,sum_amount_cat7
str,i64,i64
"""CS031414000102""",1800,558
"""CS028511000038""",1028,880
"""CS006411000021""",2373,1780
"""CS031415000177""",8951,3654
"""CS035514000115""",286,0


---

> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客 ID（customer_id）、売上日、会員申込日とともに 10 件表示せよ（sales_ymd は数値、application_date は文字列でデータを保持している点に注意）。


In [68]:
df_receipt.join(df_customer, on="customer_id", how="inner").with_columns(
    (
        pl.col("sales_ymd").cast(pl.String).str.strptime(pl.Date, "%Y%m%d")
        - pl.col("application_date").cast(pl.String).str.strptime(pl.Date, "%Y%m%d")
    ).alias("days_diff")
).select(
    pl.col("customer_id"),
    pl.col("sales_ymd"),
    pl.col("application_date"),
    pl.col("days_diff"),
)

customer_id,sales_ymd,application_date,days_diff
str,i64,i64,duration[ms]
"""CS006214000001""",20181103,20150201,1371d
"""CS008415000097""",20181118,20150322,1337d
"""CS028414000014""",20170712,20150711,732d
"""CS025415000050""",20180821,20160131,933d
"""CS003515000195""",20190605,20150306,1552d
…,…,…,…
"""CS010414000008""",20180131,20150203,1093d
"""CS004515000066""",20181217,20150617,1279d
"""CS046415000017""",20190911,20180304,556d


---

> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客 ID（customer_id）、売上日、会員申込日とともに 10 件表示せよ（sales_ymd は数値、application_date は文字列でデータを保持している点に注意）。1 ヶ月未満は切り捨てること。


---

> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客 ID（customer_id）、売上日、会員申込日とともに 10 件表示せよ（sales_ymd は数値、application_date は文字列でデータを保持している点に注意）。1 年未満は切り捨てること。


---

> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客 ID（customer_id）、売上日、会員申込日とともに 10 件表示せよ（なお、sales_ymd は数値、application_date は文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は 0 時 0 分 0 秒を表すものとする。


---

> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに 10 件表示せよ（sales_ymd は数値でデータを保持している点に注意）。


---

> P-075: 顧客データ（df_customer）からランダムに 1%のデータを抽出し、先頭から 10 件表示せよ。


---

> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに 10%のデータを層化抽出し、性別コードごとに件数を集計せよ。


---

> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から 3σ を超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は 10 件表示せよ。


---

> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客 ID が"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第 1 四分位と第 3 四分位の差である IQR を用いて、「第 1 四分位数-1.5×IQR」を下回るもの、または「第 3 四分位数+1.5×IQR」を超えるものとする。結果は 10 件表示せよ。


---

> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。


---

> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079 で確認した件数だけ減少していることも確認すること。


---

> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については 1 円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。


---

> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については 1 円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。


---

> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については 1 円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。


---

> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める 2019 年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は 0 として扱うこと。そして計算した割合が 0 超のものを抽出し、結果を 10 件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。


---

> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1 つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を 10 件表示せよ。


---

> P-086: 085 で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客 ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は 10 件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$


---

> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして 1 顧客 1 レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客 ID（customer_id）の番号が小さいものを残すこととする。


---

> P-088: 087 で作成したデータを元に、顧客データに統合名寄 ID を付与したデータを作成せよ。ただし、統合名寄 ID は以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客 ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客 ID を設定
>
> 顧客 ID のユニーク件数と、統合名寄 ID のユニーク件数の差も確認すること。


---

> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ 8:2 の割合でランダムにデータを分割せよ。


---

> P-090: レシート明細データ（df_receipt）は 2017 年 1 月 1 日〜2019 年 10 月 31 日までのデータを有している。売上金額（amount）を月次で集計し、学習用に 12 ヶ月、テスト用に 6 ヶ月の時系列モデル構築用データを 3 セット作成せよ。


---

> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が 1:1 となるようにアンダーサンプリングで抽出せよ。


---

> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。


---

> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。


---

> P-094: 093 で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |    ファイル形式     | ヘッダ有無 | 文字エンコーディング |
> | :-----------------: | :--------: | :------------------: |
> | CSV（カンマ区切り） |    有り    |        UTF-8         |
>
> ファイル出力先のパスは以下のようにすること
>
> | 出力先 |
> | :----: |
> | ./data |


---

> P-095: 093 で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |    ファイル形式     | ヘッダ有無 | 文字エンコーディング |
> | :-----------------: | :--------: | :------------------: |
> | CSV（カンマ区切り） |    有り    |        CP932         |
>
> ファイル出力先のパスは以下のようにすること。
>
> | 出力先 |
> | :----: |
> | ./data |


---

> P-096: 093 で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |    ファイル形式     | ヘッダ有無 | 文字エンコーディング |
> | :-----------------: | :--------: | :------------------: |
> | CSV（カンマ区切り） |    無し    |        UTF-8         |
>
> ファイル出力先のパスは以下のようにすること。
>
> | 出力先 |
> | :----: |
> | ./data |


---

> P-097: 094 で作成した以下形式のファイルを読み込み、データを 3 件を表示させて正しく取り込まれていることを確認せよ。
>
> |    ファイル形式     | ヘッダ有無 | 文字エンコーディング |
> | :-----------------: | :--------: | :------------------: |
> | CSV（カンマ区切り） |    有り    |        UTF-8         |


---

> P-098: 096 で作成した以下形式のファイルを読み込み、データを 3 件を表示させて正しく取り込まれていることを確認せよ。
>
> |    ファイル形式     | ヘッダ有無 | 文字エンコーディング |
> | :-----------------: | :--------: | :------------------: |
> | CSV（カンマ区切り） | ヘッダ無し |        UTF-8         |


---

> P-099: 093 で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |   ファイル形式    | ヘッダ有無 | 文字エンコーディング |
> | :---------------: | :--------: | :------------------: |
> | TSV（タブ区切り） |    有り    |        UTF-8         |
>
> ファイル出力先のパスは以下のようにすること
>
> | 出力先 |
> | :----: |
> | ./data |


---

> P-100: 099 で作成した以下形式のファイルを読み込み、データを 3 件を表示させて正しく取り込まれていることを確認せよ。
>
> |   ファイル形式    | ヘッダ有無 | 文字エンコーディング |
> | :---------------: | :--------: | :------------------: |
> | TSV（タブ区切り） |    有り    |        UTF-8         |


# これで１００本終わりです。おつかれさまでした！
